<b>Coursera Peer Review Assignment
    
Segment and Cluster Toronto Neighborhoods</b>



 <b>First Step: Use the Notebook to build the code to scrape the following Wikipedia page. </b>

In [23]:
#Import needed items
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from pandas.io.json import json_normalize  
from sklearn.cluster import KMeans

In [24]:
#scrape, removes 'non-assigned'
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    if row != [] and row[1] != "Not assigned\n":
        if "Not assigned\n" in row[2]: 
            row[2] = row[1]
        res.append(row)

df = pd.DataFrame(res, columns = ["Postal Code", "Borough", "Neighborhood"])
df.head()



,Postal Code,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


In [26]:
#clean up the /n on the fields

df["Postal Code"] = df["Postal Code"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()


,Postal Code,Borough,Neighborhood,PostalCode
0,M3A,North York,Parkwoods,M3A
1,M4A,North York,Victoria Village,M4A
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A


In [27]:
#Group those with the same postal code
df = df.groupby(["Postal Code", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
print("Shape: ", df.shape)
df.head()

Shape:  (103, 3)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b>Second Step : Get Latitude/Longitude and make a new data frame</b>

In [28]:
#geocoder did not work for me, used alternate given in class
geo_data = pd.read_csv("http://cocl.us/Geospatial_data")

geo_data.head()
#print(geo_data.shape)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
#merging the tables to make 1 new table
df_toronto = pd.merge(df, geo_data, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476
